In [1]:
from datetime import datetime
print(str(datetime.now()))
import numpy as np
# os.environ["TF_CPP_MIN_LOG_LEVEL"] ="3"
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import math

import os
import glob
import matplotlib.pyplot as plt


2022-09-29 04:55:55.071281


2022-09-29 04:55:55.264316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-29 04:55:55.264347: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# !pip install matplotlib

In [3]:
gpu_available = tf.test.is_gpu_available()
gpu_available
req_cols = ['ITEM_ID', 'USER_ID', 'CABIN_TYPE', 'USER_RESIDENCE']

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-09-29 04:55:57.452539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-29 04:55:57.452578: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-29 04:55:57.452611: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bioss-System-Product-Name): /proc/driver/nvidia/version does not exist


In [4]:
data_set_df_updated = pd.read_csv("dataset/interaction_demo.csv")
test_df = pd.read_csv("dataset/interaction_test_demo.csv")
data_set_df_updated.loc[data_set_df_updated.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
test_df.loc[test_df.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
train_df = pd.concat([data_set_df_updated, test_df], ignore_index=True)
train_df.sort_values("TIMESTAMP", ascending= False, inplace= True)

In [5]:
train_df_item_count = train_df.groupby(["ITEM_ID"]).size().reset_index(name='counts')

In [6]:
train_df_item_count["probability"]= train_df_item_count["counts"] / train_df_item_count["counts"].sum()

In [7]:
train_df = train_df.merge(train_df_item_count[["ITEM_ID","probability"]], how='left', on='ITEM_ID',)


In [8]:
item_df = train_df[["ITEM_ID"]].drop_duplicates("ITEM_ID")
item_ds = tf.data.Dataset.from_tensor_slices(item_df.to_dict("list")).batch(32)
item_df.shape

(350, 1)

In [9]:
req_cols = req_cols+["probability"]

In [10]:
train_ds =  tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list")).batch(256)

In [11]:
USER_ID_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_ID"]))))

CABIN_TYPE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["CABIN_TYPE"]))))

USER_RESIDENCE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_RESIDENCE"]))))

In [12]:
item_unique =  np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["ITEM_ID"]))))

In [13]:
# def rankL(np_rank):
#     r = int(np_rank[-1])
#     _l = 0
#     for k in range(1, r+1):
#         _l += 1./k
#     return np.float32(_l)


# """
# labels are assumed to be 1 hot encoded
# """
# def warp_loss(labels, logits):
#     # for easy broadcasting
#     labels, logits = tf.transpose(labels, [1, 0]), tf.transpose(logits, [1, 0])
#     f_y = tf.reduce_sum(logits*labels, axis=0)
#     rank = tf.reduce_sum(tf.maximum(tf.sign(1+logits-f_y), 0), axis=0)
#     diff = tf.reduce_sum(tf.maximum(1+logits-f_y, 0), axis=0)
#     with tf.control_dependencies([tf.assert_greater(rank, tf.zeros_like(rank))]):
#         return tf.py_func(rankL, [rank], tf.float32) * diff/rank

In [14]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        emb_dim = 8    
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_ID_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_ID_unique) + 1, 16),
        ])
            
        self.cabin_type_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary= CABIN_TYPE_unique, mask_token=None),  
            tf.keras.layers.Embedding(len(CABIN_TYPE_unique) + 1, emb_dim),
        ])

        self.user_residence_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_RESIDENCE_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_RESIDENCE_unique) + 1, emb_dim),
        ])
        

    def call(self, user_interation_data):
        return tf.concat([                          
            self.user_id_embedding(user_interation_data["USER_ID"]), 
            self.cabin_type_embedding(user_interation_data["CABIN_TYPE"]), 
            self.user_residence_embedding(user_interation_data["USER_RESIDENCE"]),
        ], axis=1)





In [15]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        

        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=item_unique, mask_token=None),
            tf.keras.layers.Embedding(len(item_unique) + 1, 32),
        ])


    def call(self, user_interation_data):

        return tf.concat([
            self.item_embedding(user_interation_data["ITEM_ID"])
            
            ], axis=1)

In [16]:
class TRFSRetrievalModel(tfrs.models.Model):

    def __init__(self, UserModel,ItemModel, item_ds ):
        super().__init__()



        self.query_model = tf.keras.Sequential([#,UserModel()
          UserModel(),
#           tf.keras.layers.Dense(32 , kernel_initializer= tf.keras.initializers.RandomNormal(seed=99)),  
#           tf.keras.layers.Dropout(0.2),
        ])
        

        self.candidate_model = tf.keras.Sequential([
          ItemModel(),
#           tf.keras.layers.Dense(32, kernel_initializer= tf.keras.initializers.RandomNormal(seed=1)),
#           tf.keras.layers.Dropout(0.2),
        ]) 
        
        


#         metrics = [
#           tf.keras.metrics.TopKCategoricalAccuracy(
#               k=x, name=f"factorized_top_k/top_{x}_categorical_accuracy")
#           for x in [3,5,10,15, 25]
#         ]  
        
        self.task = tfrs.tasks.Retrieval(
#             loss=warp_loss,
            num_hard_negatives=100,
            metrics=tfrs.metrics.FactorizedTopK(
            item_ds.map(self.candidate_model),
                ks= (3, 5, 10,15, 25)),
                          

        )
        
#         self.task = tfrs.tasks.Retrieval(
#             metrics=tfrs.metrics.FactorizedTopK(
#                 candidates=item_ds.map(self.candidate_model),
#                 metrics = metrics,
#                 k = 100
#             ),
#             # temperature = 0.5,
#             num_hard_negatives = 5
#         )

    def compute_loss(self, features, training= True):

        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)
        candidate_sampling_probability = features.pop("probability")
        return self.task(query_embeddings, 
        item_embeddings, 
        compute_metrics=True,
        candidate_sampling_probability = candidate_sampling_probability
        )

    def call(self, test):
        features= test.copy()
        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return query_embeddings, item_embeddings


In [17]:
test_max_index = math.floor(train_df.shape[0]*0.1)
train_split_len = train_df.shape[0] - test_max_index
data_set_tf = tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list"))
test = data_set_tf.take(test_max_index)
train = data_set_tf.skip(test_max_index).take(train_split_len)
shuffled = train.shuffle(train_split_len, seed=42, reshuffle_each_iteration=True)
cached_train = shuffled.batch(512).prefetch(4096)#train.shuffle(100_000).batch(2048)
cached_test = test.batch(512).cache()


In [18]:
train_df.shape[0], test_max_index

(34853, 3485)

In [19]:
def delete_all_check_points(fpath= 'new_amazon_check_points/*'):
    files = glob.glob(fpath)
    for f in files:
        os.remove(f)
        

In [20]:
def get_call_back_fun(K):
    delete_all_check_points()
    model_path = f"new_amazon_check_points/best_check_point_{K}k"
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        save_weights_only=True,
        monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
        mode='max',
        save_best_only=True)

    early_stoping = tf.keras.callbacks.EarlyStopping(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
                                                     mode='min',
                                                     patience=5)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy', 
                                                     factor=0.6,
                                                     #mode='min',
                                                     patience=9, 
                                                     min_lr=1e-6
    )
    return model_path, model_checkpoint_callback, early_stoping, reduce_lr

In [21]:
model_path, model_checkpoint_callback, early_stoping, reduce_lr = get_call_back_fun(25)

In [22]:
model = TRFSRetrievalModel(UserModel, ItemModel, item_ds)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01)) 

In [23]:
data = model.fit(cached_train,
          validation_data=cached_test,
          epochs=200,
          verbose=1, 
          workers=3,
          use_multiprocessing=True,
          callbacks=[model_checkpoint_callback, 
                     reduce_lr]

          )

Epoch 1/200
62/62 [==============================] - 6s 53ms/step - factorized_top_k/top_3_categorical_accuracy: 0.0408 - factorized_top_k/top_5_categorical_accuracy: 0.0801 - factorized_top_k/top_10_categorical_accuracy: 0.1535 - factorized_top_k/top_15_categorical_accuracy: 0.2196 - factorized_top_k/top_25_categorical_accuracy: 0.3236 - loss: 2886.4490 - regularization_loss: 0.0000e+00 - total_loss: 2886.4490 - val_factorized_top_k/top_3_categorical_accuracy: 0.1690 - val_factorized_top_k/top_5_categorical_accuracy: 0.2356 - val_factorized_top_k/top_10_categorical_accuracy: 0.3308 - val_factorized_top_k/top_15_categorical_accuracy: 0.4069 - val_factorized_top_k/top_25_categorical_accuracy: 0.5268 - val_loss: 1947.3022 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1947.3022 - lr: 0.0100
Epoch 2/200
62/62 [==============================] - 3s 46ms/step - factorized_top_k/top_3_categorical_accuracy: 0.1838 - factorized_top_k/top_5_categorical_accuracy: 0.2620 - factorized_top_

62/62 [==============================] - 3s 46ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8529 - factorized_top_k/top_5_categorical_accuracy: 0.9098 - factorized_top_k/top_10_categorical_accuracy: 0.9432 - factorized_top_k/top_15_categorical_accuracy: 0.9570 - factorized_top_k/top_25_categorical_accuracy: 0.9706 - loss: 1126.9419 - regularization_loss: 0.0000e+00 - total_loss: 1126.9419 - val_factorized_top_k/top_3_categorical_accuracy: 0.2241 - val_factorized_top_k/top_5_categorical_accuracy: 0.2941 - val_factorized_top_k/top_10_categorical_accuracy: 0.3882 - val_factorized_top_k/top_15_categorical_accuracy: 0.4740 - val_factorized_top_k/top_25_categorical_accuracy: 0.6009 - val_loss: 2390.5852 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2390.5852 - lr: 0.0100
Epoch 9/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8594 - factorized_top_k/top_5_categorical_accuracy: 0.9145 - factorized_top_k/top_10_cat

62/62 [==============================] - 3s 46ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8780 - factorized_top_k/top_5_categorical_accuracy: 0.9378 - factorized_top_k/top_10_categorical_accuracy: 0.9680 - factorized_top_k/top_15_categorical_accuracy: 0.9770 - factorized_top_k/top_25_categorical_accuracy: 0.9870 - loss: 1041.4865 - regularization_loss: 0.0000e+00 - total_loss: 1041.4865 - val_factorized_top_k/top_3_categorical_accuracy: 0.2135 - val_factorized_top_k/top_5_categorical_accuracy: 0.2875 - val_factorized_top_k/top_10_categorical_accuracy: 0.3857 - val_factorized_top_k/top_15_categorical_accuracy: 0.4491 - val_factorized_top_k/top_25_categorical_accuracy: 0.5681 - val_loss: 2542.7681 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2542.7681 - lr: 0.0060
Epoch 19/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8797 - factorized_top_k/top_5_categorical_accuracy: 0.9391 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 47ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8866 - factorized_top_k/top_5_categorical_accuracy: 0.9456 - factorized_top_k/top_10_categorical_accuracy: 0.9744 - factorized_top_k/top_15_categorical_accuracy: 0.9828 - factorized_top_k/top_25_categorical_accuracy: 0.9898 - loss: 1013.7032 - regularization_loss: 0.0000e+00 - total_loss: 1013.7032 - val_factorized_top_k/top_3_categorical_accuracy: 0.2198 - val_factorized_top_k/top_5_categorical_accuracy: 0.2858 - val_factorized_top_k/top_10_categorical_accuracy: 0.3773 - val_factorized_top_k/top_15_categorical_accuracy: 0.4456 - val_factorized_top_k/top_25_categorical_accuracy: 0.5590 - val_loss: 2625.4431 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2625.4431 - lr: 0.0036
Epoch 29/200
62/62 [==============================] - 3s 45ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8873 - factorized_top_k/top_5_categorical_accuracy: 0.9463 - factorized_top_k/top_10_ca

KeyboardInterrupt: 

In [27]:
model.load_weights("new_amazon_check_points/best_check_point_25k")

In [28]:
result_summary = model.evaluate(cached_test)

7/7 [==============================] - 0s 38ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2410 - factorized_top_k/top_5_categorical_accuracy: 0.3300 - factorized_top_k/top_10_categorical_accuracy: 0.4551 - factorized_top_k/top_15_categorical_accuracy: 0.5377 - factorized_top_k/top_25_categorical_accuracy: 0.6468 - loss: 2458.4392 - regularization_loss: 0.0000e+00 - total_loss: 2458.4392


In [29]:
{val:result_summary[idx] for idx, val in enumerate([3, 5, 10,15, 25]) }

{3: 0.24103300273418427,
 5: 0.329985648393631,
 10: 0.45509326457977295,
 15: 0.5377331376075745,
 25: 0.6467719078063965}

In [31]:
data.history

In [ ]:
keys = iter([key for key in data.history.keys() if key.startswith("factorized_top")])

In [ ]:
x = [i for i in range(1,201,1)]
for key in keys:
    plt.plot(x, data.history[key], "-b", label="Train")
    plt.plot(x, data.history["val_"+key], "-r", label="Test")
    plt.legend(loc="upper left")
    plt.title(key.split("/")[1])
    break

    

In [ ]:
plt.show()